In [1]:
import bz2
import numpy as np
from tqdm import tqdm_notebook as tqdm
import gzip
from heapq import heappushpop
from joblib import Parallel, delayed
import time

In [2]:
import joblib
x_trn_hash = joblib.load('10M/eps0.1/x_trn_hash.pkl')
w = joblib.load('10M/eps0.1/w.pkl')
b = joblib.load('10M/eps0.1/b.pkl')

In [3]:
y_trn = joblib.load('y_trn.pkl')

In [4]:
dataset_ids = []
dataset_vals = []
for data_id in range(10):
    st = time.time()
    dataset_val = np.load('x_trn_' + str(data_id) + '.npy')
    dataset_vals.append(dataset_val)
    print(time.time() - st)

185.55871200561523
184.48889183998108
185.77436637878418
187.37603640556335
187.73654007911682
187.07505345344543
186.7574598789215
187.71444296836853
187.8561646938324
188.4095380306244


In [5]:
class X:
    def __init__(self, data, offset):
        self.data = data
        self.offset = offset
    
    def __getitem__(self, key):
        index1 = (key + self.offset) // 1000000
        index2 = (key + self.offset) % 1000000
        return self.data[index1][index2]
    
    def __len__(self):
        l = 0
        for x in self.data:
            l += len(x)
        return l - self.offset

In [6]:
x_trn = X(dataset_vals, 1100)

In [7]:
y_tst = np.load('y_tst.npy')
y_val = np.load('y_val.npy')
x_tst = np.load('x_tst.npy')
x_val = np.load('x_val.npy')

In [8]:
x_val2 = x_val[:100]
y_val2 = y_val[:100]

In [14]:
def val_error(K, sp_gt):
    K_star = 10
    start = time.time()
    x_val_knn_approx, nns_vec = lsh.get_approx_KNN(x_val2, K_star)
    runtime_query = time.time() - start
    print(runtime_query)
    
    start = time.time()
    sp_approx = lsh.compute_approx_shapley(x_val_knn_approx, y_val2, K)
    runtime_approx_value = time.time() - start
    print('it takes %s to get appox knn value' % runtime_approx_value)
    
    sp_err_inf_val= np.linalg.norm(sp_gt - sp_approx,ord=np.inf, axis=1)
    print('max error %s'% np.percentile(sp_err_inf_val,90))
    return sp_approx

In [10]:
def test_error(K, sp_gt):
    K_star = 10
    start = time.time()
    x_tst_knn_approx, nns_vec = lsh.get_approx_KNN(x_tst, K_star)
    runtime_query = time.time() - start
    print(runtime_query)
    
    start = time.time()
    sp_approx = lsh.compute_approx_shapley(x_tst_knn_approx, y_tst, K)
    runtime_approx_value = time.time() - start
    print('it takes %s to get appox knn value' % runtime_approx_value)
    
    sp_err_inf_val= np.linalg.norm(sp_gt - sp_approx,ord=np.inf, axis=1)
    print('max error %s'% np.percentile(sp_err_inf_val,90))
    return sp_approx

In [9]:
dist_rand = np.load('10M/eps0.1/dist_rand.npy')
dist_rand = np.mean(dist_rand, axis=0)

In [12]:
from tqdm import tqdm_notebook as tqdm
def get_true_KNN(x_trn,x_tst,K):
    N = len(x_trn)
    N_tst = len(x_tst)
    x_tst_knn_gt = np.zeros((N_tst, N))
    for i_tst in tqdm(range(N_tst)):
        dist_gt = np.zeros(N)
        for i_trn in tqdm(range(N)):
            dist_gt[i_trn] = np.linalg.norm(x_trn[i_trn] - x_tst[i_tst], 2)
#             if i_trn % 100000 == 0:
#                 print(i_trn)
        x_tst_knn_gt[i_tst, :] = np.argsort(dist_gt)
    return x_tst_knn_gt.astype(int)

In [14]:
start = time.time()
x_val_knn_gt = get_true_KNN(x_trn, x_val2, 1)
val_end1 = time.time() - start
print(val_end1)


20253.718082904816


In [18]:
def equal(a, b):
    try:
        return not set.isdisjoint(a, b)
    except KeyError:
        return 0
    
def compute_single_unweighted_knn_class_shapley(x_trn,y_trn,x_tst_knn_gt,y_tst,K):
    N = len(x_trn)
    N_tst = len(x_tst_knn_gt)
    sp_gt = np.zeros((N_tst,N))
    for j in tqdm(range(N_tst)):
        sp_gt[j,x_tst_knn_gt[j,-1]] = equal(y_trn[x_tst_knn_gt[j,-1]], y_tst[j])/N
        for i in tqdm(np.arange(N-2,-1,-1)):
            sp_gt[j,x_tst_knn_gt[j,i]] = sp_gt[j,x_tst_knn_gt[j,i+1]] +\
                           (equal(y_trn[x_tst_knn_gt[j,i]], y_tst[j]) -
                            equal(y_trn[x_tst_knn_gt[j,i+1]], y_tst[j]))/K* min([K,i+1])/(i+1)
    return sp_gt

In [ ]:
K = 2
start2 = time.time()
val_sp_gt2 = compute_single_unweighted_knn_class_shapley(x_trn, y_trn, x_val_knn_gt, y_val2, K)
end2 = time.time() - start2
print(end2)

In [ ]:
K = 5
start5 = time.time()
val_sp_gt5 = compute_single_unweighted_knn_class_shapley(x_trn, y_trn, x_val_knn_gt, y_val2, K)
end5 = time.time() - start5

In [22]:
val_sp_gt2.shape

(100, 9998900)

In [23]:
val_sp_gt5.shape

(100, 9998900)

In [24]:
np.save('10M/eps0.1/val_sp_gt2.npy', val_sp_gt2)
np.save('10M/eps0.1/val_sp_gt5.npy', val_sp_gt5)

In [10]:
val_sp_gt2 = np.load('10M/eps0.1/val_sp_gt2.npy')

In [11]:
def equal(a, b):
    try:
        return not set.isdisjoint(a, b)
    except KeyError:
        return 0

import numpy as np
import pdb


def lsh_function(t,x,w,b):
    # x is 1-d array
    h = np.floor((np.dot(w,x)+b)/t).astype(int)
    return h


class LSH:
    def __init__(self,n_hash_bit,n_hash_table,x_trn,y_trn,t=0.1):
        self.n_hash_bit = n_hash_bit
        self.n_hash_table = n_hash_table
        self.t = t # width of projections
        self.x_trn = x_trn
        self.y_trn = y_trn
        self.N = len(x_trn)
        self.dim = 4096
        # draw w from a normal distribution (2-stable)
        self.w = np.random.normal(0, 1, (n_hash_table, n_hash_bit, self.dim))
        # draw b from U[0,t]
        self.b = np.random.uniform(0, self.t, (n_hash_table, n_hash_bit))
        self.x_trn_hash = [dict() for i in range(n_hash_table)]
#         for i in tqdm(range(self.N)):
#             hash_code_all = lsh_function(self.t, x_trn[i] / dist_rand, self.w, self.b)
#             for l in range(n_hash_table):
#                 hash_code_trn = '.'.join(map(str, hash_code_all[l, :]))
#                 if hash_code_trn in self.x_trn_hash[l].keys():
#                     self.x_trn_hash[l][hash_code_trn].append(i)
#                 else:
#                     self.x_trn_hash[l][hash_code_trn] = [i]
#             if i % 1000 == 0:
#                 print('build hash %s'%i)

    def get_approx_KNN(self,x_tst,K):
        N_tst = x_tst.shape[0]
        x_tst_knn = np.ones((N_tst, K)) * (-1)
        nns_len = np.zeros(N_tst)
        for i_tst in tqdm(range(N_tst)):
            nns = np.array([])
            for l in range(self.n_hash_table):
                hash_code_int = lsh_function(self.t, x_tst[i_tst] / dist_rand, self.w[l, :, :], self.b[l, :])
                hash_code_test = '.'.join(map(str, hash_code_int))
                if hash_code_test in self.x_trn_hash[l].keys():
                    nns = np.append(nns, self.x_trn_hash[l][hash_code_test])
            nns = np.unique(nns)
            nns = nns.astype(int)
            num_collide_elements = len(nns)
            if len(nns) > 0:
                dist = [np.linalg.norm(self.x_trn[i] / dist_rand - x_tst[i_tst] / dist_rand, 2) for i in nns]
                dist_min_ind = nns[np.argsort(dist)]
                if num_collide_elements < K:
                    x_tst_knn[i_tst, :num_collide_elements] = dist_min_ind[:num_collide_elements]
                else:
                    x_tst_knn[i_tst, :] = dist_min_ind[:K]
            # pdb.set_trace()
            nns_len[i_tst] = len(nns)
            if i_tst % 100 == 0:
                print('get approximate knn %s'%i_tst)
        return x_tst_knn.astype(int),nns_len


    def compute_approx_shapley(self,x_tst_knn,y_tst,K):
        N_tst,K_star = x_tst_knn.shape
        # flag_sufficient = (x_tst_knn[:,-1]>=0)
        sp_approx = np.zeros((N_tst,self.N))
        for j in tqdm(range(N_tst)):
            non_nan_index = np.where(x_tst_knn[j,:]>=0)[0]
            if len(non_nan_index)== 0:
                continue
            K_tot = non_nan_index[-1]
            if K_tot == self.N:
                sp_approx[j, x_tst_knn[j, self.N - 1]] = equal(self.y_trn[x_tst_knn[j, self.N - 1]], y_tst[j]) / self.N
            for i in np.arange(K_tot - 1, -1, -1):
                sp_approx[j, x_tst_knn[j, i]] = sp_approx[j, x_tst_knn[j, i+1]] + (
                        equal(self.y_trn[x_tst_knn[j, i]], y_tst[j]) - equal(
                    self.y_trn[x_tst_knn[j, i + 1]], y_tst[j])) / K * min([K, i + 1]) / (i + 1)



        return sp_approx

In [12]:
lsh = LSH(14,75,x_trn,y_trn,t=2.203)

In [13]:
lsh.x_trn_hash = x_trn_hash
lsh.w = w
lsh.b = b

In [15]:
val_sp_gt2_approx = val_error(2, val_sp_gt2)

get approximate knn 0

4590.901703834534



it takes 0.03622841835021973 to get appox knn value
max error 0.09082597587356664


In [16]:
del val_sp_gt2

In [17]:
val_sp_gt5 = np.load('10M/eps0.1/val_sp_gt5.npy')

In [18]:
val_sp_gt5_approx = val_error(5, val_sp_gt5)

get approximate knn 0

4624.002764940262



it takes 0.05813264846801758 to get appox knn value
max error 0.08921904947758354
